In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import torchaudio
from pathlib import Path
from copy import copy
from docx.api import Document
import soundfile as sf
import librosa
from pydub.utils import mediainfo
from tqdm.auto import tqdm
import pandas as pd

from data_utils import NeutuneSet

In [3]:
# dataset = NeutuneSet('/home/teo/userdata/nia_dataset/field_recording')
dataset = NeutuneSet('/home/teo/label-studio_files/nia_dataset/')

In [18]:
from pandas import read_table

from meta_utils import MetaCreator

meta_creator = MetaCreator('vocab.json')
# meta_creator.create_meta_for_wav(dataset.wav_list[0])
entire_meta = [meta_creator.create_meta_for_wav(wav) for wav in tqdm(sorted(dataset.wav_list))]

  0%|          | 0/498 [00:00<?, ?it/s]

Correct middle: 1 / Current middle: 2 / Current Fine: 2


In [14]:
xlsx_path = '/home/teo/label-studio_files/nia_dataset/__DATA_v4/01/metadata/fs-2-4-75-1-가민지.xlsx'
detailed_meta = pd.read_excel(xlsx_path, index_col=0)

detailed_meta = detailed_meta.T
'사용 기자재' in detailed_meta
# detailed_meta = detailed_meta.T
# detailed_meta

True

In [11]:
detailed_meta

,분류 번호,녹음/제작자 성명,녹음/제작 연/월/일,오디오 포맷,오디오 길이,제작 장소,보안 이슈,자료 제공자 정보
Unnamed: 1,1-1-2,가민지,2022.07.17,"스테레오 - wav (96k, 24bit) / mp3",91초,서산시 동문동,없음,성명: 000\n연령: 000


In [13]:
xlsx_path = '/home/teo/label-studio_files/nia_dataset/__DATA_v4/01/metadata/fs-2-4-75-1-가민지.xlsx'
detailed_meta = pd.read_excel(xlsx_path, index_col=0)
detailed_meta

,Unnamed: 1
분류 번호,1-1-2
녹음/제작자 성명,가민지
녹음/제작 연/월/일,2022.07.17
사용 기자재,Ableton Live DAW
오디오 포맷,"스테레오 - wav (96k, 24bit) / mp3"
오디오 길이,91초
제작 장소,서산시 동문동
보안 이슈,없음
자료 제공자 정보,성명: 000\n연령: 000


In [15]:
for wav_path in dataset.wav_list:
  docx_path = wav_path.parent.parent / 'metadata' / wav_path.with_suffix('.docx').name
  xlsx_path = docx_path.with_suffix('.xlsx')
  if docx_path.exists() and xlsx_path.exists():
    xlsx_path.unlink()

In [19]:
meta_creator.error_list

[{'file': 'fs-2-4-75-1-김양우.wav', 'error': 'Meta docx does not exist'},
 {'file': 'fs-2-4-75-1-김준형.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': 'fs-2-4-75-1-이동화.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': 'fs-2-4-75-1-장완석.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': 'fs-2-4-75-2-김양우.wav', 'error': 'Meta docx does not exist'},
 {'file': 'fs-2-4-75-2-김준형.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': 'fs-2-4-75-2-이동화.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': 'fs-2-4-75-2-장완석.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': 'fs-2-4-75-3-김양우.wav', 'error': 'Meta docx does not exist'},
 {'file': 'fs-2-4-75-3-김준형.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': 'fs-2-4-75-3-이동화.wav',
  'error': 'Error occured while handling the correspon

In [22]:
docx_path = '/home/teo/label-studio_files/nia_dataset/__DATA_v4/01/metadata/fs-2-4-75-2-장완석.docx'
meta_creator.read_table_from_docx(docx_path)

{'': '\n'}

In [ ]:

'''
class MetaCreator:
  def __init__(self, vocab_path):
    with open(vocab_path, 'r') as f:
      self.vocab = json.load(f)
    self.error_list = []

  def get_recording_type(self, wav_path):
    if '폴리' in wav_path:
      return '폴리'
    elif '합성' in wav_path:
      return '합성'
    elif '필드' in wav_path:
      return '필드 레코딩'

  def check_mp3_format(self, wav_path):
    mp3_path = wav_path.parent.parent / 'mp3' / wav_path.with_suffix('.mp3').name
    mp3_info = mediainfo(mp3_path)
    if int(mp3_info["sample_rate"]) != 48000:
        print(f"Error in MP3 SR: {mp3_path} is {mp3_info['sample_rate']}")
    if abs(int(mp3_info["bit_rate"]) - 320000) > 300 :
      print(f"Error in MP3 bit_rate: {mp3_path} is {mp3_info['bit_rate']}")
    return float(mp3_info["duration"])

  def check_name_is_wrong(self, wav_path):

    name = wav_path.stem
    high, middle, fine = [int(x) for x in name.split('-')[1:4]]
    voc = self.vocab[fine-1]
    assert voc["소분류_번호"] == fine
    if high != voc['대분류_번호']:
      print(f"Correct High: {voc['대분류_번호']} / Current High: {high} / Current Fine: {fine}")
      return True
    if middle != voc['중분류_번호']:
      print(f"Correct middle: {voc['중분류_번호']} / Current middle: {middle} / Current Fine: {fine}")

      return True
    return False

  def add_error(self, wav_path, error_str):
    self.error_list.append({'file': wav_path.name, 'error': error_str})

  def get_category_idx(self, wav_path):
    name = wav_path.stem
    high, middle, fine = [int(x) for x in name.split('-')[1:4]]
    return fine-1

  def create_meta_for_wav(self, wav_path):
    if isinstance(wav_path, str):
      wav_path = Path(wav_path)

    # 파일 이름 형태가 1-1-1-1.wav 이런 식이라고 가정할 때
    category_idx = self.get_category_idx(wav_path)
    
    # try:
    #   mp3_dur = self.check_mp3_format(wav_path)
    # except:
    #   print(f"Error occured while reading mp3 of {wav_path}")


    if self.check_name_is_wrong(wav_path):
      # print(f"Name is wrong for {wav_path}")
      self.add_error(wav_path, "Category is wrong")
    meta = copy(self.vocab[category_idx])
    meta['녹음 방식'] = self.get_recording_type(str(wav_path))
    meta['파일 이름'] = wav_path.name

    ob = sf.SoundFile(wav_path)
    # if abs(mp3_dur - ob.frames / ob.samplerate) > 0.2:
    #   print(f"Duration of MP3 and Wav is different: {ob.frames / ob.samplerate}, {mp3_dur}")

    meta['샘플링 레이트'] = ob.samplerate
    meta['채널 수'] = ob.channels
    meta['Bit_depth'] = int(ob.subtype.split('_')[1])
    if meta['Bit_depth'] != 24:
      # print(f"Bit-depth of {wav_path} is {meta['Bit_depth']}")
      self.add_error(wav_path, f"Bit-depth is {meta['Bit_depth']}")
    if meta['채널 수'] != 2:
      print(f"Num Channels of {wav_path} is {meta['채널 수']}")
      self.add_error(wav_path, f"Num channels is {meta['채널 수']}")
    if meta['샘플링 레이트'] != 96000:
      print(f"Sampling rate of {wav_path} is {meta['샘플링 레이트']}")
      self.add_error(wav_path, f"Sampling rate is {meta['샘플링 레이트']}")

    meta['길이'] = ob.frames / ob.samplerate # second 
    if True: #if it is field recording
      docx_path = wav_path.parent.parent / 'metadata' / wav_path.with_suffix('.docx').name
      excel_path = docx_path.with_suffix('.xlsx')
      if excel_path.exists():
        try:
          detailed_meta = pd.read_excel(excel_path)
          if meta['녹음 방식'] == "필드 레코딩":
            meta["필드 레코딩 메타"] = detailed_meta
          print(excel_path)
          meta["제목"] = detailed_meta.pop("제목")
        except:
          self.add_error(wav_path, f"Error occured while handling the corresponding xlsx")
      elif docx_path.exists():
        try:
          detailed_meta = self.read_table_from_docx(docx_path)
          pd.DataFrame(detailed_meta, index=['']).T.to_excel(docx_path.with_suffix('.xlsx'))
          if meta['녹음 방식'] == "필드 레코딩":
            meta["필드 레코딩 메타"] = detailed_meta
          meta["제목"] = detailed_meta.pop("제목")
        except Exception as e:
          self.add_error(wav_path, f"Error occured while handling the corresponding docx")
      else:
        self.add_error(wav_path, "Meta docx does not exist")
    # self.get_audio_features(ob)
    return meta

  def read_table_from_docx(self, docx_path):
    document = Document(docx_path)
    table = document.tables[0]
    table_dict = {row.cells[0].text:row.cells[1].text for row in table.rows[1:]}
    return table_dict


  def get_audio_features(self, soundfile:sf.SoundFile):
    y = soundfile.read()
    y = y.mean(1)
    spec_centroid = librosa.feature.spectral_centroid(y = y, sr=soundfile.samplerate)
    print(spec_centroid)

    return
'''

In [ ]:
import numpy as np

class AudioFeatureExtractor:
  def __init__(self):
    pass 

  def get_audio_features(self, y:np.ndarray, sr:int):
    spec = self.get_spec(y, sr)
    
    spec_centroid = self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_centroid, 'spectral centroid')
    
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_bandwidth, 'spectral bandwidth'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_contrast, 'spectral contrast'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_rolloff, 'spectral roll off'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.spectral_flatness, 'spectral flatness'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.zero_crossing_rate, 'zero crossing rate'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.rms, 'rms'))

    return spec_centroid
  
  def get_spec(self, y, sr):
    return np.abs(librosa.stft(y))
  
  def get_mean_and_std_of_audio_feature(self, y, sr, audio_feature, audio_feature_name:str):
    audio_features = audio_feature(S = y, sr = sr)
    return {f'{audio_feature_name} mean': np.mean(audio_features) , f'{audio_feature_name} std':np.std(audio_features) }
  
  def get_mean_and_std_of_audio_feature_wo_sr(self, y, audio_feature, audio_feature_name:str):
    audio_features = audio_feature(y = y)
    return {f'{audio_feature_name} mean': np.mean(audio_features) , f'{audio_feature_name} std':np.std(audio_features) }

  def read_soundfile(self, soundfile:sf.SoundFile):
    y = soundfile.read()
    y = y.mean(1)
    return y, soundfile.samplerate

  def __call__(self, soundfile:sf.SoundFile):
    y, sr = self.read_soundfile(soundfile)
    return self.get_audio_features(y, sr)

extractor = AudioFeatureExtractor()
ob = sf.SoundFile(dataset.wav_list[290])

# extractor(ob)

In [ ]:
example_meta = entire_meta[290]
example_meta['오디오 음향 특성'] = extractor(ob)

In [ ]:
with open("example_meta.json", 'w') as f:
  json.dump(example_meta, f, ensure_ascii=False)
# example_meta

In [ ]:
import pandas as pd
pd.DataFrame([(x['소분류'], x['녹음 방식'], x['파일 이름'], x['길이']) for x in entire_meta]).to_csv('length.csv')

In [ ]:
len(dataset.wav_list)

In [ ]:
[error['file'] for error in meta_creator.error_list if error['error'] == 'Error occured while handling the corresponding docx']

In [ ]:
meta_creator.error_list

In [ ]:
entire_meta[290]

In [ ]:
for meta in entire_meta:
  if '필드 레코딩 메타' in meta:
    print(meta['필드 레코딩 메타']['소리의 강도'])

In [ ]:
test = Document("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/metadata/1-2-5-1-김민규.docx")
table = test.tables[0]

for row in table.rows:
  print(row.cells[0].text, '//', row.cells[1].text)

In [ ]:
meta_creator.read_table_from_docx("/home/teo/userdata/nia_dataset/field_recording/4팀 김양우 1주차/metadata/1-1-3-1-한은광.docx")

In [ ]:
Path("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/metadata/1-2-7-6-백승렬.docx").exists()

In [ ]:
table = meta_creator.read_table_from_docx('1-1-1-1-송영재.docx')
table

In [ ]:
import pandas as pd
pd.DataFrame(table, index=['']).T.to_excel('1-1-1-1-송영재.xlsx')

In [ ]:
pd.read_excel('/home/teo/label-studio_files/nia_dataset/__DATA_v4/01/metadata/r-1-1-4-11-백승렬.xlsx')

In [ ]:
!pip install openpyxl

In [ ]:
meta_creator.vocab['idx2title']

In [ ]:
mediainfo("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/mp3/1-2-7-6-백승렬.mp3")

In [ ]:
import docx

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

test = getText('1-1-1-1-송영재.docx')

In [ ]:
from docx.api import Document

# Load the first table from your document. In your example file,
# there is only one table, so I just grab the first one.
document = Document('1-1-1-1-송영재.docx')
table = document.tables[0]

In [ ]:
data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    # Establish the mapping based on the first row
    # headers; these will become the keys of our dictionary
    if i == 0:
        keys = tuple(text)
        continue

    # Construct a dictionary for this row, mapping
    # keys to values for this row
    row_data = dict(zip(keys, text))
    data.append(row_data)
data[0]['']